<a href="https://colab.research.google.com/github/GeorgeSherif/NLP-ChatEGP/blob/main/Testing%20After%20Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Testing with the Arabic Dataset.


1.   Load the model.
2.   Load the dataset.
3.   Translate the dataset and preprocess it.
4.   Test using the model.



In [1]:
!pip install xformers transformers sentencepiece torch pygal torchvision sacremoses


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317

In [2]:
import pandas as pd
import numpy as np
import os
import warnings
import csv
import re
import seaborn as sns
import pygal as py
import torch
import torchvision
import nltk
import spacy 

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import *

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings("ignore", category=FutureWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import drive
from IPython.display import display # Allows the use of display() for DataFrames
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [5]:
import pandas as pd
import numpy as np
dfTest = pd.read_csv('/content/gdrive/MyDrive/NLP/labelled.csv' ,engine="python")

In [6]:
from transformers import MarianTokenizer, MarianMTModel
mname = "Helsinki-NLP/opus-mt-tc-big-ar-en"
tokenizer = MarianTokenizer.from_pretrained(mname)
model_translate = MarianMTModel.from_pretrained(mname)

In [7]:
translated = []
for i in range(len(dfTest)):
  input = dfTest['Sentence'][i]
  translated_tokens = model_translate.generate(**tokenizer.prepare_seq2seq_batch([input], return_tensors="pt"))
  translated_text = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
  translated.append(translated_text[0])
dfTest['English'] = translated

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [8]:
dfTest = dfTest.drop(["Sentence","id"] , axis =1)
dfTest = dfTest.rename(columns={'label': 'Sentiment', 'English':'Sentence'})
dfTest['Sentiment'] = dfTest['Sentiment'].replace(['negative','neutral','positive'],[0,1,2])
dfTest

,Sentiment,Sentence
0,0,The profits of the Gulf Livestock Company fell...
1,1,Aramex to discuss cash dividend of 16% for 2016
2,2,Al Ain Al Ahlia Insurance achieved a profit of...
3,2,Umm Al Quwain Cement’s profit increased by 30%...
4,0,The General Assembly of Fujairah Cement Indust...
...,...,...
63,1,The Dow Jones quoted an official of Ajman Isla...
64,1,Depa said in a statement on Nasdaq Dubai that ...
65,2,Dubai Islamic Insurance and Reinsurance Compan...
66,2,Gulf Investment Company’s profit in 2016 was A...


In [9]:
tokens = []
for i in range(len(dfTest)):
  tokens.append(nltk.word_tokenize(dfTest['Sentence'][i]))

In [11]:
dfTest['token'] = tokens

In [12]:
nltk_stop_words = set(stopwords.words('english'))
stopRemoved = []
# print(nltk_stop_words)
for i in range(len(dfTest)):
  filtered_sentence =[]
  for w in dfTest['token'][i]:  
    if w not in nltk_stop_words:  
        filtered_sentence.append(w)  
  stopRemoved.append(filtered_sentence)
dfTest['stop'] = stopRemoved


In [13]:
punctRem = []
punctuations="?:!.,;"
for i in range(len(dfTest)):
   removedPuncForSentence = []
   for word in dfTest['stop'][i]:
      if word not in punctuations:
          removedPuncForSentence.append(word)
   punctRem.append(removedPuncForSentence)
dfTest['punct'] = punctRem

In [14]:
finalizedForm = []
for i in range(len(dfTest)):
  concatStr = ' '
  for j in range(len(dfTest['punct'][i])):
      concatStr += ' ' + dfTest['punct'][i][j]
  finalizedForm.append(concatStr)
dfTest['bertForm'] = finalizedForm

In [15]:
dfTest = dfTest.drop(["Sentence","token", "stop",	"punct"], axis = 1)
dfTest = dfTest.rename(columns={"bertForm": "Sentence"})

In [16]:
dfTest

,Sentiment,Sentence
0,0,The profits Gulf Livestock Company fell 12.1...
1,1,Aramex discuss cash dividend 16 % 2016
2,2,Al Ain Al Ahlia Insurance achieved profit AE...
3,2,Umm Al Quwain Cement ’ profit increased 30 %...
4,0,The General Assembly Fujairah Cement Industr...
...,...,...
63,1,The Dow Jones quoted official Ajman Islamic ...
64,1,Depa said statement Nasdaq Dubai completed n...
65,2,Dubai Islamic Insurance Reinsurance Company ...
66,2,Gulf Investment Company ’ profit 2016 AED 30...


In [17]:
from transformers import BertTokenizer,BertForSequenceClassification, TextClassificationPipeline

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = 3,
                                                      id2label={0: 'negative', 1: 'neutral', 2: 'positive'},
                                                      output_attentions = False,
                                                      output_hidden_states = False).to(device)
                                                      
accuracy_per_epoch = []

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
path = F"/content/gdrive/MyDrive/NLP/BERT_ft_epoch8.model"
model.load_state_dict(torch.load(path))
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#encode Arabic Testing set
encoded_data_val_Arabic = tokenizer.batch_encode_plus( dfTest.Sentence.values,
                                                       add_special_tokens = True,
                                                       return_attention_mask = True,
                                                       return_tensors = 'pt', 
                                                       truncation=True,
                                                       padding=True,
                                                      max_length = 200)

input_ids_val_Arabic = encoded_data_val_Arabic['input_ids']
attention_masks_val_Arabic = encoded_data_val_Arabic['attention_mask']
labels_val_Arabic = torch.tensor(dfTest.Sentiment.values)


#Arabic Validation set
dataset_val_Arabic = TensorDataset(input_ids_val_Arabic, 
                                   attention_masks_val_Arabic, 
                                   labels_val_Arabic)


batch_size = 8



#Arabic validation set
dataloader_val_Arabic = DataLoader(dataset_val_Arabic,
                                   sampler = RandomSampler(dataset_val_Arabic),
                                   batch_size = 8) #since we don't have to do backpropagation for this step



In [20]:
#accuracy score
def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    #make prediction
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    true = 0
    total = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        #print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n -> {len(y_preds[y_preds==label]) / len(y_true)}')
        true = true + len(y_preds[y_preds==label])
        total = total + len(y_true)
    return true / total

def evaluate(dataloader_val):

    #evaluation mode 
    model.eval()
    #tracking variables
    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        #load into GPU
        batch = tuple(b.to(device) for b in batch)

        #define inputs
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2]}

        #compute logits
        with torch.no_grad():        
            outputs = model(**inputs)
        
        #compute loss
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        #compute accuracy
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    #compute average loss
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals



In [21]:
from tqdm import tqdm

_,predictions, true_vals = evaluate(dataloader_val_Arabic)

100%|██████████| 9/9 [00:24<00:00,  2.77s/it]


In [22]:
true_vals.shape
accuracy = accuracy_per_class(predictions, true_vals)
accuracy_per_epoch.append(accuracy)

Accuracy:11/12
 -> 0.9166666666666666
Accuracy:19/28
 -> 0.6785714285714286
Accuracy:23/28
 -> 0.8214285714285714


In [23]:
accuracy_per_epoch

[0.7794117647058824]

In [24]:
# preprocess the sentence
correct = 0
false = 0
positive = 0
negative = 0
neutral = 0
for i in range(len(dfTest)):
  row = dfTest.iloc[i]
  sentence = row['Sentence']
  label = row['Sentiment']
  input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
  attention_mask = torch.tensor([1] * input_ids.shape[1]).unsqueeze(0)

  # get the predicted sentiment label
  with torch.no_grad():
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      _, predicted_label = torch.max(outputs[0], dim=1)
  print(sentence, predicted_label,label)
  if(label == 0):
    negative = negative + 1
  elif(label == 1):
    neutral = neutral + 1
  elif(label == 2):
    positive = positive + 1
  if(predicted_label == label):
    correct = correct + 1
  else:
    false = false + 1
  print(correct,false)

  The profits Gulf Livestock Company fell 12.1 million dirhams 2010 compared 27.6 million dirhams 2009 tensor([0]) 0
1 0
  Aramex discuss cash dividend 16 % 2016 tensor([2]) 1
1 1
  Al Ain Al Ahlia Insurance achieved profit AED 2.50 million 2012 compared AED 2.53 million 2011 tensor([2]) 2
2 1
  Umm Al Quwain Cement ’ profit increased 30 % AED 7.22 million first quarter year tensor([2]) 2
3 1
  The General Assembly Fujairah Cement Industries decided distribute cash dividends shares year 2010 tensor([2]) 0
3 2
  Next Tuesday ADCB ’ Board Directors meet discuss general matters tensor([1]) 1
4 2
  Dubai Development Company ’ profit last year AED 735,000 compared AED 444,000 2013 tensor([2]) 2
5 2
  Al Ain Al Ahlia Insurance Company announced disclose financial results third quarter 2011 31/10/2011 tensor([1]) 1
6 2
  HSBC set $ 100 million fund support UAE SMEs tensor([2]) 2
7 2
  The meeting Board Directors Arab Company Heavy Industries held 17-12-2012 discuss routine work tensor([2]) 1


In [25]:
print("Accuracy of testing = ", 100 * correct / (correct + false)) 
print(negative,positive,neutral) 

Accuracy of testing =  77.94117647058823
12 28 28
